# Automation BQ mail table for candidate monetization
Extraction and preprocessing of email data. This notebook simulates the exact procedure that will follow the script to be executed daily, to create a table in BQ and follow mails for candidate monetization.

Check script in: _automation_bq_mail_

In [10]:
import pandas as pd
import numpy as np
import pandas_gbq

import queries_source
import extract_automation as extract 
import preprocess_automation as preprocess

# A. Extraction
Data imported

**EMAIL INFORMATION**
- Sendgrid events: 
    - email-opened
    - email-bounce: Pasa cuando los emails rebotan porque el usuario se desuscribió, nos marcó como Spam o el servidor de correos nos bloquea
    - email-dropped: Pasa cuando los emails rebotan porque el usuario se desuscribió o nos marcó como Spam
    - email-group_unsubscribe: Cuando el usuario se desuscribe del segmento de correos de notificación
    - email-spamreport: nos marcan como spam
    - email-unsubscribe: el usuario usó el botón de unsubscribe
- Sent events
- Clicked events

## Loading dataframes 

__Email info__

In [2]:
source_sendgrid, source_sent, source_clicked = extract.extract_data_candidate(tables = [
        'sendgrid_events',
        'sent_events',
        'clicked_events'
])

Extracting data from BQ...


__Checking loaded info__

In [3]:
source_sent.head(2)

,iduser,campaign,email_id,subject_id,ziprecruiter_job_id_array,timestamp
0,006e790a-89c2-4406-8256-0b59aba1c6cc,campaign_02,22df8f8e-43ef-48bd-9ee7-42780dc31a3b,jobs-001,"[u'ziprecruiterpaid0_cpcb9cca589-f260cbf7', u'...",2020-04-27 13:27:09+00:00
1,010a9c95-5996-4895-97a3-a20a7311f066,campaign_01,6846759f-1f69-4e4e-b65b-4bc18876dcc6,jobs-001,"[u'ziprecruiter-sponsored-vet_cpc-18c7cdac', u...",2020-04-27 13:28:55+00:00


In [4]:
source_sendgrid.head(2)

,iduser,eventname,campaign,email_id,subject_id,timestamp
0,8ec48498-1917-4384-af3c-fe51092ebcab,email-dropped,campaign_01,cd591829-0543-4001-952a-0f755a7c4d32,jobs-001,2020-04-27 21:12:27+00:00
1,8f0a5fe6-a3ec-40c9-8142-ad1a263484c7,email-dropped,campaign_01,82592cba-9e85-4ef3-9af1-a27c77b7c0a4,jobs-001,2020-04-27 21:12:42+00:00


In [5]:
source_clicked.head(2)

,iduser,campaign,email_id,subject_id,ziprecruiter_job_id,timestamp
0,a297e395-c662-465b-af6c-7fb4846022b2,campaign_01,174347e5-7e94-45cf-b1b2-fe08ccc53cf6,jobs-001,ziprecruiterpaid0_cpcb9cca589-e0c2e8e0,2020-04-27 23:00:37+00:00
1,bb9a8f75-b69f-4839-a4c8-754eba70fa70,campaign_01,10ac08ec-2e95-480f-88ce-7ec1ae8be76d,jobs-001,ziprecruiterpaid0independentsalesfbf3e610fbf3e...,2020-04-27 23:06:44+00:00


# Preprocessing data
## Emails sent

In [6]:
df_sent = preprocess.preprocess_emails_sent(source_sent, source_sendgrid)
df_sent.head(2)

Preprocessing emails sent...
Number emails sent: 241588
Number emails bounced/dropped: 3202
Number emails unsubscribed: 247
Number emails spammed: 45


,iduser,campaign,email_id,subject_id,ziprecruiter_job_id_array,timestamp,bounced_mail,unsubscribed_mail,spam_mail
0,006e790a-89c2-4406-8256-0b59aba1c6cc,campaign_02,22df8f8e-43ef-48bd-9ee7-42780dc31a3b,jobs-001,"[u'ziprecruiterpaid0_cpcb9cca589-f260cbf7', u'...",2020-04-27 13:27:09+00:00,False,False,False
1,010a9c95-5996-4895-97a3-a20a7311f066,campaign_01,6846759f-1f69-4e4e-b65b-4bc18876dcc6,jobs-001,"[u'ziprecruiter-sponsored-vet_cpc-18c7cdac', u...",2020-04-27 13:28:55+00:00,False,False,False


## Emails opened

In [7]:
df_opened = preprocess.preprocess_emails_opened(source_sendgrid)
df_opened.head(2)

Preprocessing emails opened...
Length emails opened with duplicates: 39117
Length emails opened without duplicates: 22948


,iduser,eventname,campaign,email_id,subject_id,opened_timestamp
5,506e8616-52af-4c4b-b112-67c52c8b10a4,email-open,campaign_01,2ccf52ff-0de5-4de1-8f2a-d13b89ed451c,jobs-001,2020-04-27 21:12:24+00:00
6,8a903d09-2efc-4418-ad6d-bc12d36d6e0e,email-open,campaign_01,268e47f6-c7d2-4e46-98d2-af607a2e63c4,jobs-001,2020-04-27 21:18:10+00:00


## Emails clicked

In [8]:
df_clicked = preprocess.preprocess_emails_clicked(source_clicked)
df_clicked.head(2)

Preprocessing emails clicked...
Length emails clicked with duplicates: 2982
Length emails clicked without duplicates: 1883


,iduser,email_id,num_clicks
0,0014b2df-e462-4db7-b683-6ff4ca5a7b0c,d4bc271f-fbec-41a1-bb7a-5e58f8f50f1c,1
1,00635f7d-96a0-432b-a5c2-89e067074cc8,38d4ee88-ab40-473a-9b45-af50d5dc97c1,1


## Building funnel

In [14]:
df_funnel = preprocess.build_funnel(df_sent, df_opened, df_clicked)
df_funnel.head(2)

Building funnel...
Length before joining: 241588
Length after joining: 241588


,iduser,campaign,email_id,subject_id,ziprecruiter_job_id_array,timestamp,bounced_mail,unsubscribed_mail,spam_mail,eventname,opened_timestamp,num_clicks
0,006e790a-89c2-4406-8256-0b59aba1c6cc,campaign_02,22df8f8e-43ef-48bd-9ee7-42780dc31a3b,jobs-001,"[u'ziprecruiterpaid0_cpcb9cca589-f260cbf7', u'...",2020-04-27 13:27:09+00:00,False,False,False,NaN,NaT,NaN
1,010a9c95-5996-4895-97a3-a20a7311f066,campaign_01,6846759f-1f69-4e4e-b65b-4bc18876dcc6,jobs-001,"[u'ziprecruiter-sponsored-vet_cpc-18c7cdac', u...",2020-04-27 13:28:55+00:00,False,False,False,NaN,NaT,NaN


In [15]:
drop_cols_master = ['ziprecruiter_job_id_array', 'eventname']
df_master = preprocess.build_master(df_funnel, drop_cols_master)
df_master.head(2)

Building master table...
Length master df: 241588


,iduser,campaign,email_id,subject_id,timestamp,bounced_mail,unsubscribed_mail,spam_mail,opened_timestamp,num_clicks,opened_mail,clicked_mail
0,006e790a-89c2-4406-8256-0b59aba1c6cc,campaign_02,22df8f8e-43ef-48bd-9ee7-42780dc31a3b,jobs-001,2020-04-27 13:27:09+00:00,False,False,False,NaT,NaN,False,False
1,010a9c95-5996-4895-97a3-a20a7311f066,campaign_01,6846759f-1f69-4e4e-b65b-4bc18876dcc6,jobs-001,2020-04-27 13:28:55+00:00,False,False,False,NaT,NaN,False,False


Inspecting NAN and filling them if needed

In [16]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241588 entries, 0 to 241587
Data columns (total 12 columns):
iduser               241588 non-null object
campaign             241588 non-null object
email_id             241588 non-null object
subject_id           241588 non-null object
timestamp            241588 non-null datetime64[ns, UTC]
bounced_mail         241588 non-null bool
unsubscribed_mail    241588 non-null bool
spam_mail            241588 non-null bool
opened_timestamp     15633 non-null datetime64[ns, UTC]
num_clicks           1246 non-null float64
opened_mail          241588 non-null bool
clicked_mail         241588 non-null bool
dtypes: bool(5), datetime64[ns, UTC](2), float64(1), object(4)
memory usage: 15.9+ MB


In [17]:
values_bool = dict.fromkeys(['num_clicks'], 0)
df_master = preprocess.preprocess_master(df_master, values_num_dict={}, values_bool_dict=values_bool)

Preprocessing master table...


In [18]:
df_master.head(2)

,iduser,campaign,email_id,subject_id,timestamp,bounced_mail,unsubscribed_mail,spam_mail,opened_timestamp,num_clicks,opened_mail,clicked_mail
0,006e790a-89c2-4406-8256-0b59aba1c6cc,campaign_02,22df8f8e-43ef-48bd-9ee7-42780dc31a3b,jobs-001,2020-04-27 13:27:09+00:00,False,False,False,NaT,0.0,False,False
1,010a9c95-5996-4895-97a3-a20a7311f066,campaign_01,6846759f-1f69-4e4e-b65b-4bc18876dcc6,jobs-001,2020-04-27 13:28:55+00:00,False,False,False,NaT,0.0,False,False


## Importing directly to GBQ

In [ ]:
df_master.to_gbq('miscellaneous.candidate_monetization_mails', 'merlin-pro', if_exists='replace')